# Bot que realiza mantenimiento de ventas

### Información requerida:
- Fecha (Esta es la fecha en la que el bot va a realizar el mantenimiento de ventas) con el formato AAAA-MM-DD.
- Monto inicial (El monto inicial hace referencia a el total de efectivo reportado de la fecha seleccionada).
- Productos (Basta con sacar el id o clave del producto del **BackOffice** y agregarlo a la lista de productos, el bot automaticamente y de forma random seleccionara un producto y realizara la sustitución)

### Servidores:
- Centro: 192.168.193.252
- Araucarias: 192.168.193.250
- CEDIS: 192.168.193.109
- Pruebas: 192.168.193.61

In [ ]:
# Dependencias
import requests
import random
from IPython.display import Markdown as md
import json

## Fecha y Efectivo reportado
Llena los siguientes datos con tus especificaciones.

In [ ]:
datos ={
    "fecha": "2024-08-08",
    "efectivo_inicial": 24878.30,
    "servidor": "192.168.193.61"
}

In [ ]:
corte = datos["efectivo_inicial"] / 2 # Esto es el corte al 50%

## Lista de productos
En esta lista solo debes agregar el id o clave del producto seleccionado en el **BackOffice**

In [ ]:
productos = ["034003", "042035"]
random.shuffle(productos) # Mezclar los productos

## Detalles de los productos
034003 -> Cafe en grano = $80

042035 -> Pan para llevar = $40

## Obtener Folios
El diccionario de ventas se obtiene desde la API:

*http://127.0.0.1:8000/mantenimiento/cuentas/efectivo*

Esto obtiene folios con el siguiente filtro:
- Cuentas en efectivo
- Monto mayor a $120
- No facturado

In [ ]:
# Endpoint donde se obtiene la información de las ventas en efectivo
endpoint_ventas_efectivo = f'http://{datos["servidor"]}:8000/mantenimiento/cuentas/efectivo'

# Obtener la información de las ventas en efectivo
try:
    # El JSON que se le pasa es para que filtre las ventas de un día en específico.
    ventas_json = requests.get(endpoint_ventas_efectivo, json=datos)
    print(f'Se obtuvieron las ventas en efectivo correctamente. \nTotal de folios obtenidos: {len(ventas_json.json())}')
except Exception as e:
    print(f"Error al obtener la información de las ventas en efectivo: {e}")

### Lista de Folios
Un ejemplo de la salida de ventas_json es el siguiente:


**[{"folio": 3328, "total": "248.0000"}, {"folio": 3330, "total": "253.0000"}]**


En la proxima linea de codigo:


Los folios serán guardados en una lista para poder iterar sobre ellos y poder hacer el mantenimiento de ventas.

In [ ]:
# obtener los folios de la venta
folios_crudo = [folio["folio"] for folio in ventas_json.json()]

# Mezclar los folios
random.shuffle(folios_crudo)

## Mantenimiento de ventas
**Nota:** En el siguiente bloque de código se hará la sustitución de los productos en cada una de las ventas, este proceso es automatico, recuerda que hasta este punto cuentas con todos los folios de la fecha ingresada, tienes una cantidad de efectivo inicial y un corte que sirve para parar el mantenimiento de las cuentas. Los productos de los folios serán sustituidos por los productos que estan en la lista.

### Recuerda:
- Tener respaldo de la base de datos.
- Tener un reporte de corte de caja y un reporte de cuentas desgloze.
- Hacer la sincronización.
- Validar que los folios que solicitaron facturación ya esten completos.

In [ ]:
total_afectado = 0
folios_afectados = []
for folio in folios_crudo:
    body_json ={
        "cantidad": 1,
        "idproducto": random.choice(productos), # Escoger un producto al azar (antes se mezclaron los productos)
    }
    api_mantenimiento = f'http://{datos["servidor"]}:8000/mantenimiento/actualizar/{folio}/'
    try:
        # Se manda la petición a la API para actualizar la venta
        response = requests.put(api_mantenimiento, json=body_json)
        if response.status_code == 200:
            # Cuenta de los productos que se actualizaron
            folios_afectados.append(folio)
            total_afectado += float(ventas_json.json()[folios_crudo.index(folio)]["total"])
            if body_json["idproducto"] == "034003":
                print(f"Se actualizó la venta con folio {folio} con el producto 'Café en grano 1/4'.")
                total_afectado -= 80
            if body_json["idproducto"] == "042035":
                print(f"Se actualizó la venta con folio {folio} con el producto 'Pan para llevar'.")
                total_afectado -= 40
            # Parar si ya tenemos el 50% afectado
            if total_afectado >= corte:
                print(f"Se ha llegado al corte del 50%.")
                break
        else:
            print(f"No se pudo actualizar la venta con folio {folio}.")
    except Exception as e:
        print(f"No se pudo actualizar la venta con folio {folio}.\nDetalles del error: {e}")


In [ ]:
api_desglose = f'http://{datos["servidor"]}:8000/mantenimiento/cuentas/sustituidas'
cuentas_desglose = requests.get(api_desglose, json=datos)

In [ ]:
md(f'# Reporte final\n\n'
   f'#### Ventas obtenidas: {len(ventas_json.json())} \n\n'
   f'### Se afectó el 50% de las ventas en efectivo.\n\n'
   f'#### Efectivo inicial: ${datos["efectivo_inicial"]}\n\n'
   f'#### Efectivo final: ${datos["efectivo_inicial"] - total_afectado}\n\n'
   f'#### Efectivo afectado: ${total_afectado}\n\n'
   f'#### Folios afectados: \n\n{folios_afectados}\n\n'
   f'## Cuentas desglose:\n\n'
   f'{json.dumps(cuentas_desglose.json())}\n\n'
   f'\n\n **DataBot** - {datos["fecha"]}.'
   )

# Cuentas en efectivo facturas.
Este modulo es especial para sacar los reportes de efectivo facturado.

In [ ]:
# Endpoint para obtener las ventas facturadas
api_efectivo_facturado = f'http://{datos["servidor"]}:8000/mantenimiento/cuentas/efectivo/facturado'
# Obtener las ventas facturadas
try:
    ventas_facturadas = requests.get(api_efectivo_facturado, json=datos)
    print(f'Se obtuvieron las ventas facturadas correctamente. \nTotal de folios obtenidos: {len(ventas_facturadas.json())}')
except Exception as e:
    print(f"Error al obtener la información de las ventas facturadas: {e}")

In [ ]:
# Crear una lista de folios facturados
folios_facturados = [folio["folio"] for folio in ventas_facturadas.json()]

# Contador para el total facturado
total_facturado = 0

In [ ]:
# Algortimo para imprimir los folios facturados y el total de cada uno
for folio in folios_facturados:
    print(f'Folio {folio} - Total: ${ventas_facturadas.json()[folios_facturados.index(folio)]["total"]}')
    total_facturado += float(ventas_facturadas.json()[folios_facturados.index(folio)]["total"])

In [ ]:
print(f'Total facturado: ${total_facturado}')